In [1]:
pip install bitermplus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.1/615.1 KB 9.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for bitermplus: filename=bitermplus-0.6.12-cp39-cp39-linux_x86_64.whl size=1284980 sha256=48e8c37625989bb25b01f68fde055e17320d24c339a8e8244c03d1c87fd91723
  Stored in directory: /root/.cache/pip/wheels/ba/74/cb/67f35c5f8bcd2cce3324ce724326debc6aa0dc6c6c3f2e98e2
Successfully built bitermplus


In [3]:
%pip install HanTa
%pip install lda
%pip install pyldavis
%pip install germalemma==0.1.3

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.0/15.0 MB 43.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.5/348.5 KB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 KB 9.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 26.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.1/22.1 MB 47.0 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer
# import lda
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from HanTa import HanoverTagger as ht
from nltk.corpus import stopwords
from germalemma import GermaLemma
import pyLDAvis.gensim_models
import pyLDAvis
pyLDAvis.enable_notebook()

nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
en_tagger = ht.HanoverTagger('morphmodel_en.pgz')
de_tagger = ht.HanoverTagger('morphmodel_ger.pgz')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
def clean_data():
    def remove_emojis(data):
        emoj = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                          u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                          u"\U0001F680-\U0001F6FF"  # transport & map symbols
                          u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                          u"\U00002500-\U00002BEF"  # chinese char
                          u"\U00002702-\U000027B0"
                          u"\U00002702-\U000027B0"
                          u"\U000024C2-\U0001F251"
                          u"\U0001f926-\U0001f937"
                          u"\U00010000-\U0010ffff"
                          u"\u2640-\u2642"
                          u"\u2600-\u2B55"
                          u"\u200d"
                          u"\u23cf"
                          u"\u23e9"
                          u"\u231a"
                          u"\ufe0f"  # dingbats
                          u"\u3030"
                          "]+", re.UNICODE)
        return re.sub(emoj, '', data)

    # Removing mentions and hashtags and url
    def remove_mentions_and_tags(text):
        text = re.sub(r'@\S*', '', text)
        text = re.sub(r'http\S+', '', text)
        return re.sub(r'#\S*', '', text)

    read_df = pd.read_parquet("/content/drive/MyDrive/twitter_data.parquet")
    read_df = read_df.drop_duplicates(subset=['tweet_id'])
    read_df = read_df.loc[read_df['tweet_type'] != 'retweet']
    read_df.text = read_df.text.apply(remove_emojis)
    read_df.text = read_df.text.apply(remove_mentions_and_tags)
    read_df['text'] = read_df['text'].str.replace(r'[^\w\s]+', '')  # remove punctuations
    read_df['text'] = read_df['text'].str.replace('\d+', '')  # removes number
    en_df = read_df[read_df.tweet_language == 'en']
    de_df = read_df[read_df.tweet_language == 'de']
    return en_df, de_df, read_df

def extract_adj_noun(df, tagger: ht.HanoverTagger, lang: str):

  def noun(text, tagger):
    words = nltk.word_tokenize(text)
    tokens = [word for (word, x, pos) in tagger.tag_sent(
        words, taglevel=1) if pos == 'NN']
    return tokens

  def adj(text, tagger):
    words = nltk.word_tokenize(text)
    tokens = [word for (word, x, pos) in tagger.tag_sent(
        words, taglevel=1) if pos == 'ADJ']
    return tokens

  def to_lowercase(text):
    token = []
    for i in range(len(text)):
      val = text[i].lower()
      token.append(val)
    return token

  def stop_word_removal(x, stop_words):
    return ' '.join([w for w in x if not w in stop_words])

  df['noun'] = df.text.apply(noun, tagger=tagger)
  df['adj'] = df.text.apply(adj, tagger=tagger)
  df.noun = df.noun.apply(to_lowercase)
  stop_words = stopwords.words(lang)
  stop_words.append('amp')
  df.noun = df.noun.apply(stop_word_removal, stop_words=stop_words)
  df.adj = df.adj.apply(stop_word_removal, stop_words=stop_words)
  return df


def lementize_en_text(df):
  w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
  lemmatizer = nltk.stem.WordNetLemmatizer()

  def lemmatize_text(text):
    return [lemmatizer.lemmatize(w, 'n') for w in w_tokenizer.tokenize(text)]

  def lemmatize_text_adj(text):
    return [lemmatizer.lemmatize(w, 'a') for w in w_tokenizer.tokenize(text)]

  df['lemma_noun'] = df.noun.apply(lemmatize_text)
  df['lemma_adj'] = df.adj.apply(lemmatize_text_adj)
  dict_docs = df.lemma_noun.to_dict()
  return df, dict_docs


def lementize_de_text(df):
  lemmatizer = GermaLemma()

  def lemmatize_noun(x):
    arr = []
    token = x.split()
    for i in range(len(token)):
      lemma_noun = lemmatizer.find_lemma(token[i], 'N')
      arr.append(lemma_noun)
    return arr

  def lemmatize_adj(x):
    arr = []
    token = x.split()
    for i in range(len(token)):
      lemma_noun = lemmatizer.find_lemma(token[i], 'ADJ')
      arr.append(lemma_noun)
    return arr

  df['lemma_noun'] = df.noun.apply(lemmatize_noun)
  df['lemma_adj'] = df.adj.apply(lemmatize_adj)
  dict_docs = df.lemma_noun.to_dict()
  return df, dict_docs


def lda_topic_model(df):

  def generate_tokens(tweet):
    words = []
    for i in range(len(tweet)):
      word = tweet[i]
    # using the if condition because we introduced extra spaces during text cleaning
      if word != '':
        words.append(word)
    return words

  def create_dictionary(words):
    return corpora.Dictionary(words)

  def create_document_matrix(tokens, id2word):
    corpus = []
    for text in tokens:
      corpus.append(id2word.doc2bow(text))
    return corpus

  #storing the generated tokens in a new column named 'words'
  df['tokens'] = df.lemma_noun.apply(generate_tokens)

  #passing the dataframe column having tokens as the argument
  id2word = create_dictionary(df.tokens)

  #passing the dataframe column having tokens and dictionary
  corpus = create_document_matrix(df.tokens, id2word)
  return corpus, id2word


def fetch_doc_topic(df, corpus, lda_model, ntopics):
    def get_doc_topic(corpus, model):
        doc_topic = []
        for doc in range(len(corpus)):
            try:
                doc_topic.append(
                    {'Tweet': df.iloc[doc].text, 'Hashtags': df.iloc[doc].hashtags, 'Topic_Probability': model.__getitem__(corpus[doc], eps=0)})
            except Exception as e:
                print(e)
        return doc_topic

    def calc_doc_topic_count(doc_topic, count_arr):
        for topic in doc_topic:
            prob = topic['Topic_Probability']
            big = -99999
            for p in prob:
                if p[1] > big:
                    big = p[1]
                    max_top = p[0]
            count_arr[max_top] += 1
        return count_arr

    doc_topic = get_doc_topic(corpus, lda_model)
    doc_topic_df = pd.DataFrame(doc_topic)
    count_arr = calc_doc_topic_count(doc_topic, [0]*ntopics)
    return doc_topic, doc_topic_df, count_arr


def doc_topic_pie_chart(ntopics, count_arr):
    topics = []
    for i in range(ntopics):
        topics.append(i+1)
    plt.figure(figsize=(8, 8))
    plt.pie(count_arr, labels=topics,
            autopct='%1.1f%%', textprops={'fontsize': 18})
    plt.title('Tweets distribution in topics', fontsize=20)
    #plt.legend(data_pie.tweets_count,fontsize=20)
    plt.show()


def print_beauty(df, word_topic_df):
    for i in range(6, 10):
        print('Tweet:', df.iloc[i].Tweet)
        print('Hashtags:', df.iloc[i].Hashtags)
        print('Topic_Probability:', df.iloc[i].Topic_Probability)
        print("----------------------------------------------------------------\n")

    for i in range(4):
        print("Topic ", (i+1))
        print("Most Probable Word List: ",
              word_topic_df.iloc[i].most_prob_words)
        print("-------------------------------------------------------\n")


def fetch_word_topic(lda_model, id2word):
    def get_topic_to_wordids(model):
        p = list()
        for topicid in range(model.num_topics):
            topic = model.state.get_lambda()[topicid]
            topic = topic / max(topic)  # normalize to probability dist
            p.append(topic)
        return p

    def create_dict_word_topic(id2word, word_topic):
        word_topic_dict = []
        i = 0
        for topic in word_topic:
            prob_words = []
            for word_ind in range(len(topic)):
                if (topic[word_ind] >= 0.15):
                    prob_words.append(id2word[word_ind])
            word_topic_dict.append(
                {'Topic': (i+1), 'most_prob_words': prob_words})
            i += 1
        return word_topic_dict

    word_topic = get_topic_to_wordids(lda_model)
    word_topic_dict = create_dict_word_topic(id2word, word_topic)
    word_topic_df = pd.DataFrame(word_topic_dict)
    return word_topic_dict, word_topic_df


<>:37: DeprecationWarning: invalid escape sequence \d
<>:37: DeprecationWarning: invalid escape sequence \d
<ipython-input-9-e98cc4e8346b>:37: DeprecationWarning: invalid escape sequence \d
  read_df['text'] = read_df['text'].str.replace('\d+', '')  # removes number


In [11]:
en_topics = 4
de_topics = 5

def run(df, tagger: ht.HanoverTagger, language: str, n_topics: int):
    df = extract_adj_noun(en_df, tagger, language)
    if language == "english":
        df, dict_docs = lementize_en_text(df)
    if language == "german":
        df, dict_docs = lementize_de_text(df)
    corpus, id2word = lda_topic_model(df)
    lda_model = gensim.models.ldamodel.LdaModel(
        corpus=corpus, id2word=id2word, num_topics=n_topics, random_state=42)
    doc_topic, doc_topic_df, count_arr = fetch_doc_topic(
        df, corpus, lda_model, n_topics)
    word_topic_dict, word_topic_df = fetch_word_topic(lda_model, id2word)
    return df, dict_docs, corpus, id2word, lda_model, doc_topic, doc_topic_df, count_arr, word_topic_dict, word_topic_df


en_df, de_df, read_df = clean_data()

<ipython-input-9-e98cc4e8346b>:36: FutureWarning: The default value of regex will change from True to False in a future version.
  read_df['text'] = read_df['text'].str.replace(r'[^\w\s]+', '')  # remove punctuations
<ipython-input-9-e98cc4e8346b>:37: FutureWarning: The default value of regex will change from True to False in a future version.
  read_df['text'] = read_df['text'].str.replace('\d+', '')  # removes number


In [12]:
read_df.head()

,timestamp,tweet_id,conversation_id,author_id,text,retweet_count,reply_count,like_count,quote_count,referenced_tweets,hashtags,tweet_type,tweet_language
1,2021-06-20T21:29:24.000Z,1406725899744157698,1406725899744157698,2981738470,Wenn Wirtschaftsjounalistinnen über schreiben...,9,1,62,4,[],['IchbinHanna'],original,de
5,2021-06-20T16:17:25.000Z,1406647386542325764,1406647386542325764,2981738470,,0,0,1,0,"[{'type': 'quoted', 'id': '1406620276822061057'}]","['IchbinHanna', 'PeerReview']",original,nl
11,2021-06-18T13:10:36.000Z,1405875593711964166,1405875593711964166,1132055796571877376,Thread about the cruel in German The debate...,5,0,19,0,"[{'type': 'quoted', 'id': '1405846267759054851'}]","['precarity', 'academia', 'IchbinHanna', 'Acad...",original,en
16,2021-06-18T10:21:31.000Z,1405833045224087555,1405833045224087555,242424959,Liebe und Forsa schön dass ihr euch per Umfr...,0,1,2,0,[],"['Mittelbau', 'IchBinHanna']",original,de
19,2021-06-18T08:35:29.000Z,1405806358335832065,1405806358335832065,1132055796571877376,Wichtiger Thread zu in der der durch die A...,2,0,13,0,"[{'type': 'quoted', 'id': '1405494574533984264'}]","['Machtmissbrauch', 'Wissenschaft', 'prekär', ...",original,de


In [21]:
df = pd.concat([en_df, de_df])
print(df)

                       timestamp             tweet_id      conversation_id  \
11      2021-06-18T13:10:36.000Z  1405875593711964166  1405875593711964166   
105     2021-06-12T21:15:23.000Z  1403823267115376649  1403823267115376649   
125     2021-06-11T22:32:52.000Z  1403480380632674307  1403480380632674307   
215     2021-06-10T19:07:52.000Z  1403066401611472902  1403066401611472902   
238     2021-06-10T15:04:44.000Z  1403005214974640143  1403005214974640143   
...                          ...                  ...                  ...   
308278  2022-08-15T09:31:53.000Z  1559110620972285953  1559109649995124738   
308279  2022-08-15T09:28:02.000Z  1559109649995124738  1559109649995124738   
308311  2021-08-25T04:56:31.000Z  1430393629651177473  1430093188757401618   
308326  2021-09-02T19:41:11.000Z  1433515364932067338  1433515364932067338   
308332  2022-06-28T08:32:01.000Z  1541700936128659458  1541700936128659458   

                  author_id  \
11      1132055796571877376   
1

In [22]:
texts = df['text'].str.strip().tolist()

In [23]:
import bitermplus as btm

In [24]:
X, vocabulary, vocab_dict = btm.get_words_freqs(texts)
tf = np.array(X.sum(axis=0)).ravel()
# Vectorizing documents
docs_vec = btm.get_vectorized_docs(texts, vocabulary)
docs_lens = list(map(len, docs_vec))
# Generating biterms
biterms = btm.get_biterms(docs_vec)

# INITIALIZING AND RUNNING MODEL
model = btm.BTM(
    X, vocabulary, seed=12321, T=8, M=20, alpha=50/8, beta=0.01)
model.fit(biterms, iterations=20)
p_zd = model.transform(docs_vec)

# METRICS
perplexity = btm.perplexity(model.matrix_topics_words_, p_zd, X, 8)
coherence = btm.coherence(model.matrix_topics_words_, X, M=20)
# or
perplexity = model.perplexity_
coherence = model.coherence_

# LABELS
#model.labels_
# or
btm.get_docs_top_topic(texts, model.matrix_docs_topics_)

100%|██████████| 48876/48876 [00:00<00:00, 73068.80it/s]


,documents,label
0,Thread about the cruel in German The debate...,5
1,Thread summarizing a video by the German Fede...,5
2,On job in German and its impact on life and,5
3,,0
4,German researchers without a permanent post in...,4
...,...,...
48871,Hätte nicht gedacht dass ich das nochmal sagen...,7
48872,Frisch zurück aus dem Urlaub gibts some person...,6
48873,WissZeitVG Maximaldauer erreicht\n\nEinfach un...,2
48874,beschließt neues Gesetz zur Stärkung der Berli...,2


In [25]:
!pip install tmplot

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
import tmplot as tmp
tmp.report(model=model, docs=texts)

/usr/local/lib/python3.9/dist-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
/usr/local/lib/python3.9/dist-packages/tmplot/_helpers.py:38: UserWarning: Please install "tomotopy" package to analyze its models.
Run `pip install tomotopy` in the console.
  warn(
